In [ ]:
import requests
from sketch.examples.prompt_machine import *
import json
import os
import sqlite3
import random

PM_SETTINGS["VERBOSE"] = False
SPIDER_DB_PATH = "/home/jawaugh/benchmarks/raw/spider/spider/database/"

In [ ]:
# def download_file_from_google_drive(id, destination):
#     URL = "https://docs.google.com/uc?export=download"

#     session = requests.Session()

#     response = session.get(URL, params = { 'id' : id }, stream = True)
#     token = get_confirm_token(response)

#     if token:
#         params = { 'id' : id, 'confirm' : token }
#         response = session.get(URL, params = params, stream = True)

#     save_response_content(response, destination)    

# def get_confirm_token(response):
#     for key, value in response.cookies.items():
#         if key.startswith('download_warning'):
#             return value

#     return None

# def save_response_content(response, destination):
#     CHUNK_SIZE = 32768

#     with open(destination, "wb") as f:
#         for chunk in response.iter_content(CHUNK_SIZE):
#             if chunk: # filter out keep-alive new chunks
#                 f.write(chunk)

# download_file_from_google_drive("1m_M2VOjM7Xxq0z9hxgGdYIGIKhxOIU-F", "spider_train.json")
# download_file_from_google_drive("1twy32bdOYcTY8HXrISm1vTBMMjTzI9Fa", "spider_eval.json")

data = json.load(open("spider_train.json"))
for i, row in enumerate(data):
    row['row_id'] = f"spider_train|{i}"

data_eval = json.load(open("spider_eval.json"))
for i, row in enumerate(data):
    row['row_id'] = f"spider_eval|{i}"

In [ ]:
# compare executed results

def get_result(db_name, sql):
    conn = sqlite3.connect(os.path.join(SPIDER_DB_PATH, f"{db_name}/{db_name}.sqlite"))
    conn.text_factory = lambda b: b.decode(errors="ignore")
    cur = conn.cursor()
    cur.execute(sql)
    return cur.fetchall()

In [ ]:
async def get_database_context(db_name):
    db_path = os.path.join(SPIDER_DB_PATH, f"{db_name}/{db_name}.sqlite")
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    # execute a sql query to get all the tables and the columns in the tables
    c.execute("SELECT name FROM sqlite_master WHERE type='table';")
    outputschema = ""
    # outputschema = "Example: table.column* (row1 value) *means join key\n"
    for (table,) in c.fetchall():
        # # # STYLE 2
        # outputschema += f"===== {table} =====\n"
        # c.execute(f"PRAGMA table_info({table})")
        # column_names = [column for _, column, dtype, *_ in c.fetchall()]
        # output = c.execute(f"select * from {table} limit 2").fetchall()
        # for i, column in enumerate(column_names):
        #     if len(output) == 0:
        #         outputschema += f"{column} : -empty-"
        #     elif len(output) == 1:
        #         outputschema += f"{column} : {output[0][i]}"
        #     else:
        #         outputschema += f"{column} : {output[0][i]} | {output[1][i]}"
        #     outputschema += "\n"

        # # STYLE 1
        outputschema += f"{table} ("
        fks = [(tab, fro, to) for _, _, tab, fro, to, *_ in c.execute(f"PRAGMA foreign_key_list({table})").fetchall()]
        cols_to_star = [co for _, co, _ in fks]
        c.execute(f"PRAGMA table_info({table})")
        columns = [column + ("*" if pk or column in cols_to_star else "") for _, column, dtype, _, _, pk in c.fetchall() ]
        outputschema += "|".join(columns)
        outputschema += ") \n"
        output = c.execute(f"select * from {table} limit 2").fetchall()
        if len(output) == 0:
            outputschema += "Empty Table"
        else:
            for row in output:
                outputschema += "|".join([str(x) for x in row]) + "\n"
        outputschema += "  foreign keys: " + ", ".join([f"{fro} -> {to} ({tab})" for tab, fro, to in fks]) + "\n"
        outputschema += "\n"

        # # STYLE 3
        # fks = [(tab, fro, to) for _, _, tab, fro, to, *_ in c.execute(f"PRAGMA foreign_key_list({table})").fetchall()]
        # cols_to_star = [co for _, co, _ in fks]
        # columns = [column + ("*" if pk or column in cols_to_star else "") for _, column, dtype, _, _, pk in c.execute(f"PRAGMA table_info({table})").fetchall() ]
        # output = c.execute(f"select * from {table} limit 2").fetchall()
        # outputschema += f"== {table} ==\n"
        # for i, column in enumerate(columns):
        #     if len(output) == 0:
        #         outputschema += f"{column} ()"
        #     elif len(output) == 1:
        #         outputschema += f"{column} ({output[0][i]})"
        #     else:
        #         outputschema += f"{column} ({output[0][i]}|{output[1][i]})"
        #     outputschema += "\n"
        # outputschema += "\n"
    return outputschema

get_db_context = asyncPrompt("get_database_context", get_database_context)
correct_sqlite_error = asyncGPT3Edit(
    "correct_sqlite_error",
    "Update the query to fix the error (correct the sql query between the ``` marks): [{{ error }}]",
    model_name="code-davinci-edit-001",
    temperature=0.0,
    )

# gpt3_zeroshot_sql = asyncGPT3Prompt("gpt3_zeroshot_sql", """
# Database Context
# {{ db_context }}
# SQLite Query for question [{{ question }}]:
# ```
# """, stop="```", temperature=0.0)

# gpt3_zeroshot_explain_plan = asyncGPT3Prompt("gpt3_zeroshot_explain_plan", """
# Database Context
# {{ db_context }}
# SQLite Query for question [{{ question }}]...
# 1) First we need to consider
# """, stop="```", temperature=1.0, model_name="text-davinci-002")

gpt3_zeroshot_sql_warm = asyncGPT3Prompt("gpt3_zeroshot_sql_warm", """
Database Context
{{ db_context }}
SQLite Query for question [{{ question }}]:
```""", stop="```", temperature=0.4, model_name="code-davinci-002")

def clean_sql_quick(sql):
    sql = sql.strip()
    sql = sql.replace("\n", "")
    removals = ["sqlite3", "sqlite", "sql", "SQL"]
    for removal in sorted(removals, key=lambda x: len(x), reverse=True):
        if sql.startswith(removal):
            sql = sql[len(removal):]
    return sql

async def get_sql_for_question_multi(db_name, text_in, top_n=5):
    db_context = await get_db_context(db_name)
    sqls = await asyncio.gather(*[gpt3_zeroshot_sql_warm(db_context=db_context, question=text_in) for _ in range(top_n)])
    sqls = list(set([clean_sql_quick(sql) for sql in sqls]))

    async def run_sql_for_result(sql):
        trials = 0
        while trials < 4:
            try:
                return (sql, get_result(db_name, sql))
            except Exception as e:
                last_prompt = gpt3_zeroshot_sql_warm.get_prompt(db_context=db_context, question=text_in) + sql + '```'
                response = await correct_sqlite_error(error=str(e), input=last_prompt)
                try:
                    sql = response.split('```')[1]
                    sql = clean_sql_quick(sql)
                except:
                    sql = ""
                trials += 1
        return (sql, None)
        
    results = await asyncio.gather(*[run_sql_for_result(sql) for sql in sqls])

    answers = {json.dumps(res): set() for _, res in results if res is not None}
    for sql, res in results:
        if res is not None and sql:
            answers[json.dumps(res)].add(sql)
    if len(answers) == 0:
        return "FAILED TO GENERATE SQL"

    top_answers = sorted(answers.values(), key=len, reverse=True)[0]
    return next(iter(top_answers))

get_sql_from_text_multi = asyncPrompt("get_sql_from_text_multi", get_sql_for_question_multi)

In [ ]:
async def row_runner(row, verbose=False):
    sql_answers = {
        'query_answer': row['query'],
        'gpt3_answer': await get_sql_from_text_multi(row['db_id'], row['text_in']),
    }
    sql_executed = {}
    for k, query in sql_answers.items():
        newrow = k.split('_')[0]+'_data'
        try:
            sql_executed[k.split('_')[0]+'_data'] = get_result(row['db_id'], query)
        except Exception as e:
            sql_executed[k.split('_')[0]+'_data'] = str(e)

    if not isinstance(sql_executed['query_data'], list):
        print("Gold failed to execute")

    execution_equals = {}
    for k, v in sql_executed.items():
        newrow = k.replace('data', 'is_equal')
        if not isinstance(v, list):
            execution_equals[newrow] = False
        execution_equals[newrow] = v == sql_executed['query_data']

    execution_is_superset = {}
    for k, v in sql_executed.items():
        newrow = k.replace('data', 'is_superset')
        if not isinstance(v, list):
            execution_is_superset[newrow] = False
        if len(v) != len(sql_executed['query_data']):
            execution_is_superset[newrow] = False
        else:
            execution_is_superset[newrow] = all([set(x) >= set(y) for (x, y) in zip(v, sql_executed['query_data'])])
    
    execution_is_same_set_of_rows = {}
    for k, v in sql_executed.items():
        newrow = k.replace('data', 'is_same_set_of_rows')
        if not isinstance(v, list):
            execution_is_same_set_of_rows[newrow] = False
        execution_is_same_set_of_rows[newrow] = set(v) == set(sql_executed['query_data'])
    return sql_answers | sql_executed | execution_equals | execution_is_superset | execution_is_same_set_of_rows

In [ ]:
# await row_runner(data[1101])

In [ ]:
# [(i, x) for i, x in enumerate(data) if x['question'] == 'What are the first and last names of the customers with the 10 cheapest invoices?']
# thing = random.sample(data, 1)[0]
# rowid = random.randint(0, len(data))
# thing = data[rowid]

In [ ]:
# print(thing['question'])
# print(thing['db_id'])

In [ ]:
import pandas as pd
import time

results = []
for i in random.sample(range(len(data)), 50):
    thing = data[i]
    print(i, thing['question'], thing['db_id'])
    result = await row_runner(thing)
    result |= {'row_number': i, 'question': thing['question'], 'db_id': thing['db_id']}
    results.append(result)
    df = pd.DataFrame(results)
    print(df['query_is_equal'].sum())
    print(df[['gpt3_is_equal', 'gpt3_is_superset', 'gpt3_is_same_set_of_rows']].sum())
    time.sleep(10.0)
# import pandas as pd


In [ ]:
# async def get_prompttext_for_id(row_id):
#     db_name, text_in = data[row_id]['db_id'], data[row_id]['question']
#     db_context = await get_db_context(db_name)
#     return gpt3_zeroshot_sql.get_prompt(db_context=db_context, question=text_in)

# print(await get_prompttext_for_id(2566))

In [ ]:
# get_result('products_gen_characteristics', "SELECT product_name FROM Products WHERE color_code != (SELECT color_code FROM Ref_Colors WHERE color_description = 'white') AND product_category_code NOT IN (SELECT product_category_code FROM Ref_Product_Categories WHERE unit_of_measure = 'Handful');")

In [ ]:
(df.gpt3_is_superset.sum()+(df.gpt3_is_same_set_of_rows.sum()-df.gpt3_is_equal.sum())) / len(df)

In [ ]:
df.to_excel("trial6.xlsx")

In [ ]:
# !pip install openpyxl